In [1]:
import pyspark as ps    # for the pyspark suite
import os               # for environ variables in Part 3

%load_ext autoreload
%autoreload 2

spark = (ps.sql.SparkSession.builder
            .appName("fraud study")
            .getOrCreate())

In [2]:
import pandas as pd

dfp = pd.read_json('data/data.json')

In [3]:
## Targets, Dropping acct_type 
fraud_accts = set(['fraudster_event', 'fraudster', 'fraudster_att'])
new_df = dfp.copy()
new_df['fraud'] = dfp['acct_type'].apply(lambda x: 1 if x in fraud_accts else 0)
new_df.drop('acct_type', axis=1, inplace=True)

In [11]:
#from bs4 import BeautifulSoup

In [10]:
#type(new_df.description[0])

str

In [4]:
df = spark.createDataFrame(new_df)

In [5]:
df.printSchema()

#print(df.count())

root
 |-- approx_payout_date: long (nullable = true)
 |-- body_length: long (nullable = true)
 |-- channels: long (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- delivery_method: double (nullable = true)
 |-- description: string (nullable = true)
 |-- email_domain: string (nullable = true)
 |-- event_created: long (nullable = true)
 |-- event_end: long (nullable = true)
 |-- event_published: double (nullable = true)
 |-- event_start: long (nullable = true)
 |-- fb_published: long (nullable = true)
 |-- gts: double (nullable = true)
 |-- has_analytics: long (nullable = true)
 |-- has_header: double (nullable = true)
 |-- has_logo: long (nullable = true)
 |-- listed: string (nullable = true)
 |-- name: string (nullable = true)
 |-- name_length: long (nullable = true)
 |-- num_order: long (nullable = true)
 |-- num_payouts: long (nullable = true)
 |-- object_id: long (nullable = true)
 |-- org_desc: string (nullable = true)
 |-- org_fa

In [6]:
df_corpus = df.select('description', 'fraud')

df_corpus.printSchema()

root
 |-- description: string (nullable = true)
 |-- fraud: long (nullable = true)



In [8]:
#print(df_corpus.count())

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o110.count

In [7]:
from pyspark.sql.functions import count

res_test = df_corpus.groupBy("fraud").agg(count("fraud"))
#res_test.printSchema()

classes_count = dict(res_test.collect())
print("class representation: {}".format(classes_count))

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34449)
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/sql/dataframe.py", line 533, in collect

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:34449)

In [ ]:
balanced_classsize = min(classes_count[1.0], classes_count[5.0], 10000)
print("using limit size: {}".format(balanced_classsize))